<a href="https://colab.research.google.com/github/Dabi1021/solar_activity/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM, GRU, Dropout, Embedding
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import data
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle
import os

2023-10-28 18:21:07.723801: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-28 18:21:07.725709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-28 18:21:07.769430: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-28 18:21:07.770383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-28 18:21:08.684176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Using TensorFlow backend


/tmp/ipykernel_1905984/3452742943.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
os.getcwd()

'/home/competition/data/Code_collection'

In [ ]:
filepath = "/home/competition/data/"
date = "20231030"

In [ ]:
# 시계열 데이터를 sequence로 변환
def to_sequence(data, scaler, sequence_length):
    data = scaler.fit_transform(data.values.reshape(-1, 1))
    sequences_X, sequences_y = [], []

    for i in range(len(data) - (sequence_length * 2)):
        sequences_X.append(data[i:i+sequence_length])
        sequences_y.append(data[i+(sequence_length * 2)])

    X = np.array(sequences_X)
    y = np.array(sequences_y)

    X = X.reshape(-1, sequence_length, 1)
    y = y.reshape(-1, 1)
    return X, y

def to_sequence_x(data, scaler, sequence_length):
    data = scaler.fit_transform(data.values.reshape(-1, 1))
    sequences_X = []

    for i in range(len(data) - sequence_length):
        sequences_X.append(data[i:i+sequence_length])

    X = np.array(sequences_X)

    X = X.reshape(-1, sequence_length, 1)
    return X

In [ ]:
# Train, test 생성
sequence_length = 30
imputer = 'hot_deck'
scaler = MinMaxScaler()

df_test = pd.read_csv(f'{filepath}data/ai_competition_test.csv')
df_train = pd.read_csv(f'{filepath}data/filled_data_{imputer}.csv')
print(df_test.isna().sum())
train = df_train['flux']
test = df_test['flux']

X_train, y_train = to_sequence(train, scaler, sequence_length)
X_test = to_sequence_x(test, scaler, sequence_length)

print(X_train.shape, y_train.shape)
print(X_test.shape)

date    0
flux    0
dtype: int64
(5419, 30, 1) (5419, 1)
(30, 30, 1)


In [ ]:
# 모델 빌더 함수 정의
def build_model(hp):
    model = keras.Sequential()

    rnn_type = hp.Choice('rnn_type', values=['SimpleRNN', 'LSTM', 'GRU'])

    if rnn_type == 'SimpleRNN':
        rnn_layer = layers.SimpleRNN
    elif rnn_type == 'LSTM':
        rnn_layer = layers.LSTM
    elif rnn_type == 'GRU':
        rnn_layer = layers.GRU
    else:
        raise ValueError("Invalid RNN type.")

    # RNN 층
    hp_units = hp.Int('rnn_units', min_value=16, max_value=256, step=16)
    hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'elu'])
    hp_regularizer = hp.Choice('regularizer', values=['l1', 'l2'])
    if hp_regularizer == 'l1' or hp_regularizer == 'l2':
        hp_lambda = hp.Float('lambda', min_value=1e-6, max_value=1e-2, sampling='log')
        regularizer = getattr(tf.keras.regularizers, hp_regularizer)(l=hp_lambda)
    else:
        regularizer = None
    hp_initializer = hp.Choice('initializer', values=['he_uniform', 'glorot_uniform', 'lecun_uniform'])
    model.add(rnn_layer(units=hp_units, activation=hp_activation, kernel_regularizer=regularizer, kernel_initializer=hp_initializer))

    # Dense 층
    hp_dense_units = hp.Int('dense_units', min_value=16, max_value=256, step=16)
    model.add(layers.Dense(units=hp_dense_units, activation='relu'))

    model.add(layers.Dense(1))

    # 학습률 조절
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mean_squared_error')
    return model

# Bayesian Optimization 설정
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=10,  # 시도할 실험 횟수 설정
    num_initial_points=5,  # 초기 실험 횟수 설정
    directory='my_dir',
    project_name=f'{imputer}_0.3_comparison_bayesian_opt'
)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=False)

# Bayesian Optimization 실행
#tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20)

Reloading Tuner from my_dir/hot_deck_0.3_comparison_bayesian_opt/tuner0.json


In [ ]:
# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print("RNN Type:", best_hps.get('rnn_type'))
print("RNN Units:", best_hps.get('rnn_units'))
print("Activation:", best_hps.get('activation'))
print("Regularizer:", best_hps.get('regularizer'))
if best_hps.get('regularizer') == 'l1' or best_hps.get('regularizer') == 'l2':
    print("Lambda:", best_hps.get('lambda'))
print("Initializer:", best_hps.get('initializer'))
print("Dense Units:", best_hps.get('dense_units'))
print("Learning Rate:", best_hps.get('learning_rate'))

Best Hyperparameters:
RNN Type: SimpleRNN
RNN Units: 16
Activation: tanh
Regularizer: l2
Lambda: 1e-06
Initializer: glorot_uniform
Dense Units: 16
Learning Rate: 0.001


In [ ]:
# 최적 모델 평가
best_model = tuner.get_best_models(num_models=1)[0]
val_loss = best_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")

best_model.summary()

36/36 [==============================] - 0s 2ms/step - loss: 0.0095
Validation Loss: 0.00949205830693245
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 16)                288       
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 577 (2.25 KB)
Trainable params: 577 (2.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0.0001, patience = 10, mode = 'auto')
best_model.fit(X_train, y_train, epochs=1000, batch_size=32, callbacks=[early_stopping])


Epoch 1/1000
119/119 [==============================] - 2s 5ms/step - loss: 0.0194
Epoch 2/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0194
Epoch 3/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0190
Epoch 4/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0190
Epoch 5/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0189
Epoch 6/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0190
Epoch 7/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0186
Epoch 8/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0189
Epoch 9/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0186
Epoch 10/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0195
Epoch 11/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0189
Epoch 12/1000
119/119 [==============================] - 1s 5ms/step - loss: 0.0188
E

In [ ]:
y_pred_test = best_model.predict(X_test)
y_pred_test = scaler.inverse_transform(y_pred_test)
val_loss = best_model.evaluate(X_val, y_val)

result = pd.DataFrame({"date" : list(range(1, 31)), "flux" : y_pred_test.reshape(-1)})
result.to_excel(f'{filepath}final_results_review/{round(val_loss, 4)}_{imputer}_rnn_{date}.xlsx', index=False)
with open(f'{filepath}final_results_review/model_{round(val_loss, 4)}_{imputer}_rnn_{date}.pickle','wb') as fw:
    pickle.dump(f'{filepath}final_results_review/model_{round(val_loss, 4)}_{imputer}_rnn_{date}.pickle', fw)

36/36 [==============================] - 0s 2ms/step - loss: 0.0095


In [ ]:
with open(f'{filepath}final_results_review/model_0.0315_hot_deck_rnn_20231027.pickle', 'rb') as f:
    model = pickle.load(f)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 256)               4352      
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 5761 (22.50 KB)
Trainable params: 5761 (22.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0.0001, patience = 10, mode = 'auto')
model.fit(X_train, y_train, epochs=1000, batch_size=32, callbacks=[early_stopping])


Epoch 1/1000
119/119 [==============================] - 2s 8ms/step - loss: 0.0190
Epoch 2/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0182
Epoch 3/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0183
Epoch 4/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0183
Epoch 5/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0188
Epoch 6/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0186
Epoch 7/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0182
Epoch 8/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0180
Epoch 9/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0180
Epoch 10/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0178
Epoch 11/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0178
Epoch 12/1000
119/119 [==============================] - 1s 8ms/step - loss: 0.0178
E

In [ ]:
val_loss = model.evaluate(X_val, y_val)
y_pred_test = model.predict(X_test)
y_pred_test = scaler.inverse_transform(y_pred_test)
result = pd.DataFrame({"date" : list(range(1, 31)), "flux" : y_pred_test.reshape(-1)})
result.to_excel(f'{filepath}final_results_review/{round(val_loss, 4)}_{imputer}_rnn_{date}.xlsx', index=False)

1/1 [==============================] - 0s 18ms/step


In [ ]:
result.to_excel(f'{filepath}final_results_review/{round(val_loss, 4)}_{imputer}_rnn_{date}.xlsx', index=False)
with open(f'{filepath}final_results_review/model_{round(val_loss, 4)}_{imputer}_rnn_{date}.pickle','wb') as fw:
    pickle.dump(f'{filepath}final_results_review/model_{round(val_loss, 4)}_{imputer}_rnn_{date}.pickle', fw)

In [ ]:
def Model1(hiddenlayer, X_train, y_train, X_test, rnn_unit, den_unit, epochs, batch_size, scaler, lambd):
    early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0.0001, patience = 10, mode = 'auto')
    regularizer=tf.keras.regularizers.L1(lambd)
    #initializer=tf.keras.regularizers.GlorotUniform()
    # model 구성
    model = Sequential()
    model.add(hiddenlayer(units=rnn_unit, input_shape=(X_train.shape[-2], X_train.shape[-1]),
                          activation='tanh', kernel_regularizer=regularizer,kernel_initializer='glorot_uniform'))
    model.add(Dense(units=den_unit, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss='mean_squared_error')

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

    # 학습
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

    # train 평가
    y_pred = model.predict(X_val)

    y_pred = scaler.inverse_transform(y_pred)
    y_val = scaler.inverse_transform(y_val)
    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print("Root Mean Squared Error:", rmse)

    y_pred_test = model.predict(X_test)
    y_pred_test = scaler.inverse_transform(y_pred_test)

    result = pd.DataFrame({"date" : list(range(1, 31)), "flux" : y_pred_test.reshape(-1)})

    return model, result, round(rmse, 2)

In [ ]:
model, result, rmse = Model1(GRU, X_train, y_train, X_test,
                             rnn_unit=16, den_unit=256, epochs=1000, batch_size=32, scaler=scaler, lambd=1e-06)

Epoch 1/1000
136/136 [==============================] - 3s 9ms/step - loss: 0.0189
Epoch 2/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0175
Epoch 3/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0172
Epoch 4/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0174
Epoch 5/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0171
Epoch 6/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0171
Epoch 7/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0169
Epoch 8/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0170
Epoch 9/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0169
Epoch 10/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0168
Epoch 11/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0168
Epoch 12/1000
136/136 [==============================] - 1s 9ms/step - loss: 0.0169
E

In [ ]:
result.to_excel(f'{filepath}final_results_review/{rmse}_{imputer}_gru_{date}.xlsx', index=False)
with open(f'{filepath}final_results_review/model_{rmse}_{imputer}_gru_{date}.pickle','wb') as fw:
    pickle.dump(best_model, fw)